# TRAVAUX PRATIQUES
### COURS DE MACHINE LEARNING - UNIVERSITE VIRTUELLE DU BURKINA FASO - MASTER FD & IA
+ ETUDIANT : NANA SIDWENDLUIAN ROMARIC
+ ENSEIGNANT : MADAME BIRBA ELIANE

### 1. Chargement des données

In [1]:
#importing pyspark
import pyspark

#importing sparksession
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.classification import RandomForestClassifier

In [2]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.master("local").appName("tp").getOrCreate()

Constatant les resultats mitigés obtenus dans le fichier précédent, nous avons rétiré certaines lignes du dataset qui contenaient des anomalies. Il s'agit notamment des lignes où:
* la variable EDUCATION a des valeurs plus grandes que 4
* la variable MARRIAGE a des valeurs 0 (ZÉRO)
* les variables PAY_0,PAY_1,PAY_2,....,PAY_6 ont des valeurs -2

In [3]:
datadft_bis = spark.read.format("csv").options(header=True,inferSchema=True).load("data/ccdefault-bis.csv")

### 2. Analyse exploratoire

In [4]:
datadft_bis.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

In [5]:
datadft_bis.count()

53104

In [6]:
datadft_bis.where("ID is null").count()

29946

In [7]:
datadft_bis.where("ID is not null").count()

23158

In [8]:
datadft_bis_clean=datadft_bis.where("ID is not null")

In [9]:
datadft_bis_clean.count()

23158

In [12]:
datadft_bis_clean.describe(datadft_bis_clean.columns).show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|                ID|         LIMIT_BAL|               SEX|         EDUCATION|          MARRIAGE|              AGE|             PAY_0|              PAY_2|              PAY_3|             PAY_4|             PAY_5|              PAY_6|        BILL_AMT1|        BILL_AMT2|        BILL_AMT3|         BILL_AMT4|         BILL_AMT5|         BILL_AMT6|         PAY_AMT1|          PAY_AMT2|          PAY_AMT3|          PAY_AMT4|          PAY_AMT5|          PAY_AMT6|           DEFAULT|
+-------+------------------+------

In [13]:
datadft_bis_clean.describe("LIMIT_BAL", "BILL_AMT1","PAY_AMT1","BILL_AMT2","PAY_AMT2").show()

+-------+------------------+-----------------+-----------------+-----------------+------------------+
|summary|         LIMIT_BAL|        BILL_AMT1|         PAY_AMT1|        BILL_AMT2|          PAY_AMT2|
+-------+------------------+-----------------+-----------------+-----------------+------------------+
|  count|             23158|            23158|            23158|            23158|             23158|
|   mean|156289.99395457294|61282.07107694965|6075.015243112532|59663.90612315399| 6121.004836341653|
| stddev|127579.85957421701|77602.34303734612|16902.04758187008|75193.40105589147|20302.039638491944|
|    min|             10000|          -165580|                0|           -67526|                 0|
|    max|           1000000|           964511|           873552|           983931|           1227082|
+-------+------------------+-----------------+-----------------+-----------------+------------------+



In [14]:
datadft_bis_clean.describe("LIMIT_BAL", "AGE").show()

+-------+------------------+-----------------+
|summary|         LIMIT_BAL|              AGE|
+-------+------------------+-----------------+
|  count|             23158|            23158|
|   mean|156289.99395457294|35.24509888591415|
| stddev|127579.85957421701|9.291428745301381|
|    min|             10000|               21|
|    max|           1000000|               79|
+-------+------------------+-----------------+



In [15]:
datadft_bis_clean.groupBy("SEX").count().orderBy(asc("count")).show() # SEXE ? HOMME=1 FEMME=2

+---+-----+
|SEX|count|
+---+-----+
|  1| 9459|
|  2|13699|
+---+-----+



In [16]:
datadft_bis_clean.groupBy("DEFAULT").count().orderBy(asc("count")).show() # DÉFAUT DE PAIEMENT ? OUI=1 NON=0

+-------+-----+
|DEFAULT|count|
+-------+-----+
|      1| 5362|
|      0|17796|
+-------+-----+



In [17]:
datadft_bis_clean.groupBy(['DEFAULT','SEX']).count().orderBy(asc("DEFAULT")).show() 
# repartition de la variable cible en fonction du sexe

+-------+---+-----+
|DEFAULT|SEX|count|
+-------+---+-----+
|      0|  1| 7081|
|      0|  2|10715|
|      1|  2| 2984|
|      1|  1| 2378|
+-------+---+-----+



In [18]:
datadft_bis_clean.groupBy(['DEFAULT','EDUCATION']).count().orderBy(asc("DEFAULT")).show() 
# repartition de la variable cible en fonction du niveau d'instruction

+-------+---------+-----+
|DEFAULT|EDUCATION|count|
+-------+---------+-----+
|      0|        0|    8|
|      0|        1| 6085|
|      0|        2| 8672|
|      0|        3| 2962|
|      0|        4|   69|
|      1|        2| 2847|
|      1|        1| 1479|
|      1|        3| 1033|
|      1|        4|    3|
+-------+---------+-----+



In [19]:
datadft_bis_clean.groupBy(['DEFAULT','MARRIAGE']).count().orderBy(asc("DEFAULT")).show() 
# repartition de la variable cible en fonction du statut matrimonial

+-------+--------+-----+
|DEFAULT|MARRIAGE|count|
+-------+--------+-----+
|      0|       1| 7792|
|      0|       2| 9809|
|      0|       3|  195|
|      1|       2| 2724|
|      1|       1| 2564|
|      1|       3|   74|
+-------+--------+-----+



In [20]:
datadft_bis_clean.groupBy(['DEFAULT','AGE']).count().orderBy(asc("DEFAULT")).show()

+-------+---+-----+
|DEFAULT|AGE|count|
+-------+---+-----+
|      0| 42|  452|
|      0| 27|  918|
|      0| 39|  566|
|      0| 31|  708|
|      0| 71|    3|
|      0| 28|  868|
|      0| 56|  104|
|      0| 50|  224|
|      0| 22|  344|
|      0| 58|   62|
|      0| 67|   11|
|      0| 40|  491|
|      0| 57|   76|
|      0| 32|  699|
|      0| 60|   31|
|      0| 73|    1|
|      0| 65|   17|
|      0| 70|    6|
|      0| 48|  275|
|      0| 25|  722|
+-------+---+-----+
only showing top 20 rows



In [21]:
datadft_bis_clean.groupBy(['DEFAULT','LIMIT_BAL']).count().orderBy(desc("DEFAULT")).orderBy(desc("count")).show()

+-------+---------+-----+
|DEFAULT|LIMIT_BAL|count|
+-------+---------+-----+
|      0|    50000| 2137|
|      0|    20000| 1070|
|      0|    80000|  946|
|      0|    30000|  867|
|      0|   200000|  811|
|      1|    50000|  790|
|      0|   150000|  626|
|      0|   100000|  620|
|      1|    20000|  593|
|      0|   180000|  567|
|      0|    60000|  537|
|      1|    30000|  522|
|      0|   140000|  488|
|      0|    70000|  480|
|      0|   500000|  458|
|      0|   130000|  448|
|      0|   210000|  431|
|      0|   120000|  430|
|      0|   230000|  429|
|      0|   360000|  410|
+-------+---------+-----+
only showing top 20 rows



In [22]:
datadft_bis_clean.createOrReplaceTempView("dataView")
spark.sql("SELECT DEFAULT, avg(LIMIT_BAL) AS BALANCE FROM dataView GROUP BY DEFAULT ORDER BY BALANCE DESC").show()

+-------+------------------+
|DEFAULT|           BALANCE|
+-------+------------------+
|      0|168451.11260957518|
|      1|115928.32525177173|
+-------+------------------+



In [23]:
datadft_bis_clean

DataFrame[ID: int, LIMIT_BAL: int, SEX: int, EDUCATION: int, MARRIAGE: int, AGE: int, PAY_0: int, PAY_2: int, PAY_3: int, PAY_4: int, PAY_5: int, PAY_6: int, BILL_AMT1: int, BILL_AMT2: int, BILL_AMT3: int, BILL_AMT4: int, BILL_AMT5: int, BILL_AMT6: int, PAY_AMT1: int, PAY_AMT2: int, PAY_AMT3: int, PAY_AMT4: int, PAY_AMT5: int, PAY_AMT6: int, DEFAULT: int]

### 3. Preparation des données
#### Renommons la colonne DEFAULT en label

In [24]:
# datadft_bis = datadft.withColumn("ID","LIMIT_BAL","SEX","EDUCATION","MARRIAGE","AGE","PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6","DEFAULT")
renamedDatadft_bis_clean = datadft_bis_clean.withColumnRenamed("DEFAULT","label")

In [25]:
renamedDatadft_bis_clean.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    1|
|  3|    90000|  2|        2|       2| 34|    0|    0|    0|    0|    0|    0|    29239|    14027|    13559|    14331|    14948|    15549|    1518|    1500|    1000|    1000|    1000|    5000|    0|
|  4|

In [26]:
#  colonne des etiquettes
colLabel = "label"

#  colonne numerique
colNum = [col for col in renamedDatadft_bis_clean.columns if col!= colLabel]

In [27]:
colNum

['ID',
 'LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

In [28]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

va =  VectorAssembler().setInputCols(colNum).setOutputCol("to_be_scaled_features")

featuredDatadft_bis_clean = va.transform(renamedDatadft_bis_clean)

scaler =  StandardScaler().setInputCol("to_be_scaled_features").setOutputCol("features")

dataset_bis = scaler.fit(featuredDatadft_bis_clean).transform(featuredDatadft_bis_clean).select("features", "label")

dataset_bis.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[2.32168783618879...|    1|
|[3.48253175428319...|    0|
|[4.64337567237759...|    0|
|[5.80421959047199...|    0|
|[6.96506350856639...|    0|
+--------------------+-----+
only showing top 5 rows



### 4. Application des modèles

In [29]:
trainSetb, testSetb = dataset_bis.randomSplit([0.8,0.2])

In [30]:
trainSetb.count()

18494

In [31]:
testSetb.count()

4664

In [32]:
trainSetb.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(24,[0,1,2,3,4,5,...|    0|
|(24,[0,1,2,3,4,5,...|    0|
|(24,[0,1,2,3,4,5,...|    0|
|(24,[0,1,2,3,4,5,...|    0|
|(24,[0,1,2,3,4,5,...|    0|
+--------------------+-----+
only showing top 5 rows



### 4.1 Logistic Regression

In [33]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lr_b = LogisticRegression(maxIter=100, regParam=0.0001, elasticNetParam=0.1)
lrModel_b = lr_b.fit(trainSetb)

In [34]:
# trainingSummary = lrModel.summary
print("Coefficients: " + str(lrModel_b.coefficients))
print("Intercept: " + str(lrModel_b.intercept))

Coefficients: [-0.02094330588103058,-0.17998491808473413,-0.04570017072716318,-0.03772318212736771,-0.08207283798231185,0.05980937514460258,0.6760420221760369,0.11032980661501554,0.08180686709890893,0.04774788768784148,0.07102923985889595,0.07978566469835958,-0.3711038032003112,0.2841025868205868,0.002833425120911844,0.049224513407792644,0.015627669327713645,-0.015931441855878616,-0.15447222157692014,-0.14494741912990822,-0.007457465858492866,-0.05608250525945112,-0.047916706055974104,0.02873570956534439]
Intercept: -0.928546893254043


In [35]:
summary = lrModel_b.summary
print("Training set areaUnderROC:",summary.areaUnderROC)
summary.roc.show()
summary.pr.show()

Training set areaUnderROC: 0.7524071908449781
+--------------------+--------------------+
|                 FPR|                 TPR|
+--------------------+--------------------+
|                 0.0|                 0.0|
|4.907459338194055E-4|0.002600472813238...|
|0.001051598429613...|0.004964539007092199|
|0.001542344363432...|0.007565011820330969|
|0.001752664049355...|0.011111111111111112|
|0.002243409983174425|0.013711583924349883|
|0.002383623107122...|0.017494089834515367|
|0.002734155916993...| 0.02056737588652482|
|0.003084688726864...| 0.02364066193853428|
|0.003295008412787437|0.027186761229314422|
|0.003715647784632...|0.030023640661938536|
|0.003996074032529445| 0.03333333333333333|
|0.004136287156477846|0.037115839243498816|
|0.004416713404374649| 0.04042553191489362|
|0.004837352776219...| 0.04326241134751773|
|0.005047672462142457| 0.04680851063829787|
| 0.00532809871003926|0.050118203309692674|
|0.005748738081884464|0.052955082742316785|
|0.005959057767807066|0.056501

In [36]:
# make predictions on the test data
lr_predictions_b = lrModel_b.transform(testSetb)
lr_predictions_b.select("prediction", "label", "features").show(25)


lr_evaluator_b = BinaryClassificationEvaluator()
print('Test Area Under ROC', lr_evaluator_b.evaluate(lr_predictions_b))

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|[2.32168783618879...|
|       0.0|    0|[0.00139301270171...|
|       0.0|    0|[0.00185735026895...|
|       0.0|    0|[0.00336644736247...|
|       0.0|    0|[0.00359861614609...|
|       0.0|    0|[0.00429512249694...|
|       0.0|    0|[0.00464337567237...|


### 4.2 Decision Tree

In [40]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

dt_b2 = DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features").setMaxDepth(8)

dtModel_b2 = dt_b2.fit(trainSetb)

# make predictions on the test data
dt_predictions_b2 = dtModel_b2.transform(testSetb)
dt_predictions_b2.select("prediction", "label", "features").show(25)

# evaluate the model
dt_evaluator_b2 = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(dt_evaluator_b2.evaluate(dt_predictions_b2, {dt_evaluator_b2.metricName: "areaUnderROC"})))

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|[2.32168783618879...|
|       0.0|    0|[0.00139301270171...|
|       0.0|    0|[0.00185735026895...|
|       0.0|    0|[0.00336644736247...|
|       0.0|    0|[0.00359861614609...|
|       0.0|    0|[0.00429512249694...|
|       0.0|    0|[0.00464337567237...|


### 4.3 Random Forest

In [41]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = "features", labelCol = "label")
rfModel_b = rf.fit(trainSetb)
rf_predictions_b = rfModel_b.transform(testSetb)
rf_predictions_b.select("prediction", "label", "features").show(25)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|[2.32168783618879...|
|       0.0|    0|[0.00139301270171...|
|       0.0|    0|[0.00185735026895...|
|       0.0|    0|[0.00336644736247...|
|       0.0|    0|[0.00359861614609...|
|       0.0|    0|[0.00429512249694...|
|       0.0|    0|[0.00464337567237...|


In [42]:
rf_evaluator2 = BinaryClassificationEvaluator(labelCol="label")
accuracy = rf_evaluator2.evaluate(rf_predictions_b)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.7793560340791312
Test Error = 0.22064396592086877


In [43]:
rf_evaluator_b = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(rf_evaluator_b.evaluate(rf_predictions_b, {rf_evaluator_b.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.7793560340791312
